In [35]:
import os
import ba_tools
import numpy as np
import pandas as pd
from ds_util.qubole import run_hive, run_presto
from ba_tools.utils import dump_to_excel
from datetime import datetime, timedelta
from functools import lru_cache
import shopper_analysis_queries as sql
from ba_tools.utils import id_input_validation
import re
import ssl
from itertools import compress

In [36]:
def exec_presto(query, return_data=True, cluster='presto-consumer-insights-dev', **kwargs):
    return run_presto(query=query, return_data=return_data, na_values='\\N')

In [64]:
def volumetrics(**params):

    columns = ['segment','past_year_penn','current_year_penn','year_over_year_change_penn']
    hh_penn = pd.DataFrame(columns=columns)

    total_penn = ['total']
    
    py = params['py']
    cy = params['cy']
    total_pop_py = params['total_pop_py']
    total_pop_cy = params['total_pop_cy']
    seg_dict = params['seg_dict']
    
    py_penn = len(py.customer_id.unique())/total_pop_py
    total_penn.append(py_penn)
    cy_penn = len(cy.customer_id.unique())/total_pop_cy
    total_penn.append(cy_penn)
    change_penn = ((cy_penn-py_penn)/py_penn)
    total_penn.append(change_penn)
    penn_df = pd.DataFrame([total_penn],columns=columns)

    hh_penn = hh_penn.append(penn_df,ignore_index=True)

    for c,v in seg_dict.items():
        for v in v:
            p = [v]
            df = py[py[c] == v]
            df1 = cy[cy[c] == v]
            py_penn = len(df.customer_id.unique())/total_pop_py
            p.append(py_penn)
            cy_penn = len(df1.customer_id.unique())/total_pop_cy
            p.append(cy_penn)
            change = ((cy_penn-py_penn)/py_penn)
            p.append(change)
            penn = pd.DataFrame([p],columns=columns)
            hh_penn = hh_penn.append(penn,ignore_index=True)

    cols = ['segment','past_year_avg_buy_rate','current_year_avg_buy_rate','past_year_median_buy_rate','current_year_idean_buy_rate','year_over_year_change_avg_buy_rate','year_over_year_change_median_buy_rate']
    br = pd.DataFrame(columns=cols)

    total_br = ['total']

    py_br = py.groupby('customer_id').price.sum()
    py_avg_br = py_br.mean()
    py_med_br = py_br.median()
    total_br.append(py_avg_br)
    total_br.append(py_med_br)
    cy_br = cy.groupby('customer_id').price.sum()
    cy_avg_br = cy_br.mean()
    cy_med_br = cy_br.median()
    total_br.append(cy_avg_br)
    total_br.append(cy_med_br)
    avg_br_change = ((cy_avg_br-py_avg_br)/py_avg_br)
    total_br.append(avg_br_change)
    med_br_change = ((cy_med_br-py_med_br)/py_med_br)
    total_br.append(med_br_change)
    br_df = pd.DataFrame([total_br],columns=cols)

    br = br.append(br_df,ignore_index=True)
    for c,v in seg_dict.items():
        for v in v:
            p = [v]
            df = py[py[c] == v]
            df1 = cy[cy[c] == v]
            py_br = df.groupby('customer_id').price.sum()
            py_avg_br = py_br.mean()
            py_med_br = py_br.median()
            p.append(py_avg_br)
            p.append(py_med_br)
            cy_br = df1.groupby('customer_id').price.sum()
            cy_avg_br = cy_br.mean()
            cy_med_br = cy_br.median()
            p.append(cy_avg_br)
            p.append(cy_med_br)
            avg_br_change = ((cy_avg_br-py_avg_br)/py_avg_br)
            p.append(avg_br_change)
            med_br_change = ((cy_med_br-py_med_br)/py_med_br)
            p.append(med_br_change)
            br_df = pd.DataFrame([p],columns=cols)

            br = br.append(br_df,ignore_index=True)

    volume_overview = hh_penn.merge(br,on='segment')
    volume_overview['avg_previous_year_volume'] = volume_overview['past_year_penn']*volume_overview['past_year_avg_buy_rate']*125000000
    volume_overview['avg_current_year_volume'] = volume_overview['current_year_penn']*volume_overview['current_year_avg_buy_rate']*125000000
    volume_overview['avg_year_over_year_volume_change'] = ((volume_overview['avg_current_year_volume']-volume_overview['avg_previous_year_volume'])/volume_overview['avg_previous_year_volume'])
    volume_overview['median_previous_year_volume'] = volume_overview['past_year_penn']*volume_overview['past_year_median_buy_rate']*125000000
    volume_overview['median_current_year_volume'] = volume_overview['current_year_penn']*volume_overview['current_year_median_buy_rate']*125000000
    volume_overview['median_year_over_year_volume_change'] = ((volume_overview['median_current_year_volume']-volume_overview['median_previous_year_volume'])/volume_overview['median_previous_year_volume'])

    volume_overview = volume_overview.transpose.reset_index(drop=True)
    return volume_overview

In [38]:
def brand_loyalty(**params):
    
    ps_br = volume_overview.transpose.reset_index(drop=True)
    brand_buy_rate = ps_br[['segment','current_year_avg_buy_rate','current_year_median_buy_rate']]
    cols = ['segment','avg_brand_buy_rate','median_brand_buy_rate']
    brand_buy_rate.columns = cols
    columns = ['segment','avg_cat_buy_rate','median_cat_buy_rate','average_share','median_share']
    category_buy_rate = pd.DataFrame(columns=columns)

    cond = cy_cat.customer_id.isin(py.customer_id)
    filtered_cat = cy_cat[cond]
    l = ['total']
    cat_br = filter_cat.groupby('customer_id').price.sum()
    avg_cat_br = cat_br.mean()
    med_cat_br = cat_br.median()
    l.append(avg_cat_br)
    l.append(med_cat_br)
    avg_share = brand_buy_rate[brand_buy_rate['segment'] == 'total'].avg_brand_buy_rate.sum() / avg_cat_br
    median_share = brand_buy_rate[brand_buy_rate['segment'] == 'total'].median_brand_buy_rate.sum() / med_cat_br
    l.append(avg_share)
    l.append(med_share)
    cat_df = pd.DataFrame(l,columns=columns)
    category_buy_rate = category_buy_rate.append(cat_df,ignore_index=True)

    for c,v in seg_dict:
        for v in v:
            l = [v]
            df = cy[cy[k] == v]
            cond = cy_cat.customer_id.isin(df.customer_id)
            filtered_cat = df[cond]
            cat_br = filter_cat.groupby('customer_id').price.sum()
            avg_cat_br = cat_br.mean()
            med_cat_br = cat_br.median()
            l.append(avg_cat_br)
            l.append(med_cat_br)
            avg_share = df[df['segment'] == v].avg_brand_buy_rate.sum() / avg_cat_br
            median_share = df[df['segment'] == v].median_brand_buy_rate.sum() / med_cat_br
            l.append(avg_share)
            l.append(med_share)
            cat_df = pd.DataFrame(l,columns=columns)
            category_buy_rate = category_buy_rate.append(cat_df,ignore_index=True)

    loyalty = brand_buy_rate.merge(category_buy_rate,on='segment')
    loyalty = loyalty.transpose().reset_index(drop=True)

    return loyalty

In [39]:
def trial_repeat(**params):

    df = cy
    df['purchase_time_filled'] = pd.to_datetime(df.receipt_created_at)
    df['delta'] = df.groupby('customer_id').purchase_time_filled.transform(pd.Series.diff)
    df1 = df.groupby('customer_id').receipt_id.count().reset_index().groupby('receipt_id').customer_id.count().reset_index()
    n = df1.customer_id.sum()
    n = ['Base Size',n]
    one = df1[df1.receipt_id == 1].customer_id.sum() / df1.customer_id.sum()
    one = ['1 purhcase',one]
    repeat = df1[df1.receipt_id >= 2].customer_id.sum() / df1.customer_id.sum()
    repeat = ['Repeat purchasers',repeat]
    two = df1[df1.receipt_id == 2].customer_id.sum() / df1.customer_id.sum()
    two = ['2 purchases',two]
    three = df1[df1.receipt_id == 3].customer_id.sum() / df1.customer_id.sum()
    three = ['3 purchases',three]
    four = df1[df1.receipt_id >= 4].customer_id.sum() / df1.customer_id.sum()
    four = ['4+ purchases',four]
    two_s = df1[df1.receipt_id == 2].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    two_s = ['2 purchases repeaters',two_s]
    three_s = df1[df1.receipt_id == 3].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    three_s = ['3 purchases repeaters',three]
    four_s = df1[df1.receipt_id >= 4].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    four_s = ['4+ purchases repeaters',four_S]
    avg_pc = df.delta[df['delta']>'1 days 00:00:00'].mean().days
    avg_pc = ['Average Purchase Cycle',avg_pc]
    med_pc = df.delta[df['delta']>'1 days 00:00:00'].median().days
    med_pc = ['MedianPurchase Cycle',med_pc]
    units = df.groupby('customer_id').receipt_item_id.count().to_frame().reset_index()
    trips = df.groupby('customer_id').receipt_id.count().to_frame().reset_index()
    trip_units = trips.merge(units,on='customer_id')
    avg_un_tot = trip_units.receipt_item_id.mean()
    avg_un_tot = ['Avg Units Total',avg_un_tot]
    med_un_tot = trip_units.receipt_item_id.median()
    med_un_tot = ['Median Units Total',med_un_tot]
    avg_un_tot = trip_units[trip_units.receipt_id >= 2].receipt_item_id.mean()
    avg_un_rep = ['Avg Units Repeaters',avg_un_rep]
    med_un_rep = trip_units[trip_units.receipt_id >= 2].receipt_item_id.median()
    med_un_rep = ['Median Units Repeaters',med_un_rep]

    data_total = [n,one,repeat,two,three,four,mean_pc,median_pc,avg_un_tot,med_un_tot,avg_un_rep,med_un_rep,two_s,three_s,four_s]
    cols=['Metric','Total Perfect Current Year']
    trial_and_repeat = pd.DataFrame(columns=cols)

    for x in data_total:
        d = pd.DataFrame([x],columns=cols)
        trial_and_repeat = trial_and_repeat.append(d,ignore_index=True)

    for c,v in seg_dict:
        for v in v:
            df = cy[cy[c] == v]
            df['purchase_time_filled'] = pd.to_datetime(df.receipt_created_at)
            df['delta'] = df.groupby('customer_id').purchase_time_filled.transform(pd.Series.diff)
            df1 = df.groupby('customer_id').receipt_id.count().reset_index().groupby('receipt_id').customer_id.count().reset_index()
            n = df1.customer_id.sum()
            n = ['Base Size',n]
            one = df1[df1.receipt_id == 1].customer_id.sum() / df1.customer_id.sum()
            one = ['1 purhcase',one]
            repeat = df1[df1.receipt_id >= 2].customer_id.sum() / df1.customer_id.sum()
            repeat = ['Repeat purchasers',repeat]
            two = df1[df1.receipt_id == 2].customer_id.sum() / df1.customer_id.sum()
            two = ['2 purchases',two]
            three = df1[df1.receipt_id == 3].customer_id.sum() / df1.customer_id.sum()
            three = ['3 purchases',three]
            four = df1[df1.receipt_id >= 4].customer_id.sum() / df1.customer_id.sum()
            four = ['4+ purchases',four]
            two_s = df1[df1.receipt_id == 2].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            two_s = ['2 purchases repeaters',two_s]
            three_s = df1[df1.receipt_id == 3].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            three_s = ['3 purchases repeaters',three]
            four_s = df1[df1.receipt_id >= 4].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            four_s = ['4+ purchases repeaters',four_S]
            avg_pc = df.delta[df['delta']>'1 days 00:00:00'].mean().days
            avg_pc = ['Average Purchase Cycle',avg_pc]
            med_pc = df.delta[df['delta']>'1 days 00:00:00'].median().days
            med_pc = ['MedianPurchase Cycle',med_pc]
            units = df.groupby('customer_id').receipt_item_id.count().to_frame().reset_index()
            trips = df.groupby('customer_id').receipt_id.count().to_frame().reset_index()
            trip_units = trips.merge(units,on='customer_id')
            avg_un_tot = trip_units.receipt_item_id.mean()
            avg_un_tot = ['Avg Units Total',avg_un_tot]
            med_un_tot = trip_units.receipt_item_id.median()
            med_un_tot = ['Median Units Total',med_un_tot]
            avg_un_tot = trip_units[trip_units.receipt_id >= 2].receipt_item_id.mean()
            avg_un_rep = ['Avg Units Repeaters',avg_un_rep]
            med_un_rep = trip_units[trip_units.receipt_id >= 2].receipt_item_id.median()
            med_un_rep = ['Median Units Repeaters',med_un_rep]

            data_total = [n,one,repeat,two,three,four,mean_pc,median_pc,avg_un_tot,med_un_tot,avg_un_rep,med_un_rep,two_s,three_s,four_s]
            col_name = v+' Current Year'
            columns=['Metric',col_name]
            for x in data_total:
                d = pd.DataFrame([x],columns=columns)
                trial_and_repeat = trial_and_repeat.merge(d,on='Metric')

    df = py
    df['purchase_time_filled'] = pd.to_datetime(df.receipt_created_at)
    df['delta'] = df.groupby('customer_id').purchase_time_filled.transform(pd.Series.diff)
    df1 = df.groupby('customer_id').receipt_id.count().reset_index().groupby('receipt_id').customer_id.count().reset_index()
    n = df1.customer_id.sum()
    n = ['Base Size',n]
    one = df1[df1.receipt_id == 1].customer_id.sum() / df1.customer_id.sum()
    one = ['1 purhcase',one]
    repeat = df1[df1.receipt_id >= 2].customer_id.sum() / df1.customer_id.sum()
    repeat = ['Repeat purchasers',repeat]
    two = df1[df1.receipt_id == 2].customer_id.sum() / df1.customer_id.sum()
    two = ['2 purchases',two]
    three = df1[df1.receipt_id == 3].customer_id.sum() / df1.customer_id.sum()
    three = ['3 purchases',three]
    four = df1[df1.receipt_id >= 4].customer_id.sum() / df1.customer_id.sum()
    four = ['4+ purchases',four]
    two_s = df1[df1.receipt_id == 2].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    two_s = ['2 purchases repeaters',two_s]
    three_s = df1[df1.receipt_id == 3].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    three_s = ['3 purchases repeaters',three]
    four_s = df1[df1.receipt_id >= 4].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
    four_s = ['4+ purchases repeaters',four_S]
    avg_pc = df.delta[df['delta']>'1 days 00:00:00'].mean().days
    avg_pc = ['Average Purchase Cycle',avg_pc]
    med_pc = df.delta[df['delta']>'1 days 00:00:00'].median().days
    med_pc = ['MedianPurchase Cycle',med_pc]
    units = df.groupby('customer_id').receipt_item_id.count().to_frame().reset_index()
    trips = df.groupby('customer_id').receipt_id.count().to_frame().reset_index()
    trip_units = trips.merge(units,on='customer_id')
    avg_un_tot = trip_units.receipt_item_id.mean()
    avg_un_tot = ['Avg Units Total',avg_un_tot]
    med_un_tot = trip_units.receipt_item_id.median()
    med_un_tot = ['Median Units Total',med_un_tot]
    avg_un_tot = trip_units[trip_units.receipt_id >= 2].receipt_item_id.mean()
    avg_un_rep = ['Avg Units Repeaters',avg_un_rep]
    med_un_rep = trip_units[trip_units.receipt_id >= 2].receipt_item_id.median()
    med_un_rep = ['Median Units Repeaters',med_un_rep]

    data_total = [n,one,repeat,two,three,four,mean_pc,median_pc,avg_un_tot,med_un_tot,avg_un_rep,med_un_rep,two_s,three_s,four_s]
    cols=['Metric','Total Perfect Previous Year']
    trial_and_repeat = pd.DataFrame(columns=cols)

    for x in data_total:
        d = pd.DataFrame([x],columns=cols)
        trial_and_repeat = trial_and_repeat.merge(d,on='Metric')

    for c,v in seg_dict:
        for v in v:
            df = py[py[c] == v]
            df['purchase_time_filled'] = pd.to_datetime(df.receipt_created_at)
            df['delta'] = df.groupby('customer_id').purchase_time_filled.transform(pd.Series.diff)
            df1 = df.groupby('customer_id').receipt_id.count().reset_index().groupby('receipt_id').customer_id.count().reset_index()
            n = df1.customer_id.sum()
            n = ['Base Size',n]
            one = df1[df1.receipt_id == 1].customer_id.sum() / df1.customer_id.sum()
            one = ['1 purhcase',one]
            repeat = df1[df1.receipt_id >= 2].customer_id.sum() / df1.customer_id.sum()
            repeat = ['Repeat purchasers',repeat]
            two = df1[df1.receipt_id == 2].customer_id.sum() / df1.customer_id.sum()
            two = ['2 purchases',two]
            three = df1[df1.receipt_id == 3].customer_id.sum() / df1.customer_id.sum()
            three = ['3 purchases',three]
            four = df1[df1.receipt_id >= 4].customer_id.sum() / df1.customer_id.sum()
            four = ['4+ purchases',four]
            two_s = df1[df1.receipt_id == 2].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            two_s = ['2 purchases repeaters',two_s]
            three_s = df1[df1.receipt_id == 3].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            three_s = ['3 purchases repeaters',three]
            four_s = df1[df1.receipt_id >= 4].customer_id.sum() / df1[df1.receipt_id >= 2].customer_id.sum()
            four_s = ['4+ purchases repeaters',four_S]
            avg_pc = df.delta[df['delta']>'1 days 00:00:00'].mean().days
            avg_pc = ['Average Purchase Cycle',avg_pc]
            med_pc = df.delta[df['delta']>'1 days 00:00:00'].median().days
            med_pc = ['MedianPurchase Cycle',med_pc]
            units = df.groupby('customer_id').receipt_item_id.count().to_frame().reset_index()
            trips = df.groupby('customer_id').receipt_id.count().to_frame().reset_index()
            trip_units = trips.merge(units,on='customer_id')
            avg_un_tot = trip_units.receipt_item_id.mean()
            avg_un_tot = ['Avg Units Total',avg_un_tot]
            med_un_tot = trip_units.receipt_item_id.median()
            med_un_tot = ['Median Units Total',med_un_tot]
            avg_un_tot = trip_units[trip_units.receipt_id >= 2].receipt_item_id.mean()
            avg_un_rep = ['Avg Units Repeaters',avg_un_rep]
            med_un_rep = trip_units[trip_units.receipt_id >= 2].receipt_item_id.median()
            med_un_rep = ['Median Units Repeaters',med_un_rep]

            data_total = [n,one,repeat,two,three,four,mean_pc,median_pc,avg_un_tot,med_un_tot,avg_un_rep,med_un_rep,two_s,three_s,four_s]
            col_name = v+' Previous Year'
            columns=['Metric',col_name]
            for x in data_total:
                d = pd.DataFrame([x],columns=columns)
                trial_and_repeat = trial_and_repeat.merge(d,on='Metric')

    return trial_and_repeat

In [40]:
def basket_overview(**params):
    
    df = basket
    
    cond = df.receipt_id.isin(cy.receipt_id)
    t_df = df[cond]
    
    total_spend = t_df.groupby('receipt_id').price.sum()
    total_units = t_df.groupby('receipt_id').receipt_item_id.count()
    avg_total_spend = total_spend.mean()
    avg_total_spend = ['Average Trip Spend',avg_total_spend]
    median_total_spend = total_spend.median()
    median_total_spend = ['Median Trip Spend',median_total_spend]
    avg_total_units = total_units.mean()
    avg_total_units = ['Average Trip Units',avg_total_units]
    median_total_units = total_units.median()
    median_total_units = ['Median Trip Units',median_total_units]
    avg_item_price = avg_total_price/avg_total_units
    avg_item_price = ['Average Price per Item',avg_item_price]
    median_item_price = median_total_price/median_total_units
    median_item_price = ['Median Price per Item',median_item_price]
        
    brand_dollars = cy.groupby('receipt_id').price.sum()
    avg_brand_spend = brand_dollars.mean()
    avg_brand_spend = ['Average Brand Spend',avg_brand_spend]
    median_brand_spend = brand_dollars.median()
    median_brand_spend = ['Median Brand Spend',median_brand_spend]
    brand_units = cy.groupby('receipt_id').receipt_item_id.count()
    avg_brand_units = brand_units.mean()
    avg_brand_units = ['Average Brand Units',avg_brand_units]
    median_brand_units = brand_units.median()
    median_brand_units = ['Median Brand Units',median_brand_units]
    
    cond_1 = cy_cat.receipt_id.isin(cy.receipt_id)
    total_cat = cy_cat[cond_1]
    cat_spend = total_cat.groupby('receipt_id').price.sum()
    cat_units = total_cat.groupby('receipt_id').receipt_item_id.count()
    avg_cat_spend = cat_spend.mean()
    avg_cat_spend = ['Average Category Spend',avg_cat_spend]
    median_cat_spend = cat_spend.median()
    median_cat_spend = ['Median Category Spend',median_cat_spend]
    avg_cat_units = cat_units.mean()
    avg_cat_units = ['Average Category Units',avg_cat_units]
    median_cat_units = cat_units.median()
    median_cat_units = ['Median Category Units',median_cat_units]
    
    data_columns = [avg_total_spend,avg_total_units,median_total_spend,median_total_units,avg_item_price,median_item_price,avg_brand_spend,avg_brand_units,median_brand_spend,median_brand_units,avg_cat_spend,avg_cat_units,median_cat_spend,median_cat_units]
    
    cols = ['Metric','Total']
    basket_overview = pd.DataFrame(columns = cols)
    
    for i in data_columns:
        d = pd.DataFrame([i],columns=cols)
        basket_overview = basket_overview.append(d,ignore_index=True)
    
    for c,v in seg_dict:
        for v in v:
            filtered = cy[cy[c] == v]
            b_cond = cy.receipt_id.isin(filtered.receipt_id)
            b_df = cy[b_cond]
            cond = df.receipt_id.isin(b_df.receipt_id)
            t_df = df[cond]

            total_spend = t_df.groupby('receipt_id').price.sum()
            total_units = t_df.groupby('receipt_id').receipt_item_id.count()
            avg_total_spend = total_spend.mean()
            avg_total_spend = ['Average Trip Spend',avg_total_spend]
            median_total_spend = total_spend.median()
            median_total_spend = ['Median Trip Spend',median_total_spend]
            avg_total_units = total_units.mean()
            avg_total_units = ['Average Trip Units',avg_total_units]
            median_total_units = total_units.median()
            median_total_units = ['Median Trip Units',median_total_units]
            avg_item_price = avg_total_price/avg_total_units
            avg_item_price = ['Average Price per Item',avg_item_price]
            median_item_price = median_total_price/median_total_units
            median_item_price = ['Median Price per Item',median_item_price]

            brand_dollars = b_df.groupby('receipt_id').price.sum()
            avg_brand_spend = brand_dollars.mean()
            avg_brand_spend = ['Average Brand Spend',avg_brand_spend]
            median_brand_spend = brand_dollars.median()
            median_brand_spend = ['Median Brand Spend',median_brand_spend]
            brand_units = b_df.groupby('receipt_id').receipt_item_id.count()
            avg_brand_units = brand_units.mean()
            avg_brand_units = ['Average Brand Units',avg_brand_units]
            median_brand_units = brand_units.median()
            median_brand_units = ['Median Brand Units',median_brand_units]

            cond_1 = cy_cat.receipt_id.isin(b_df.receipt_id)
            total_cat = cy_cat[cond_1]
            cat_spend = total_cat.groupby('receipt_id').price.sum()
            cat_units = total_cat.groupby('receipt_id').receipt_item_id.count()
            avg_cat_spend = cat_spend.mean()
            avg_cat_spend = ['Average Category Spend',avg_cat_spend]
            median_cat_spend = cat_spend.median()
            median_cat_spend = ['Median Category Spend',median_cat_spend]
            avg_cat_units = cat_units.mean()
            avg_cat_units = ['Average Category Units',avg_cat_units]
            median_cat_units = cat_units.median()
            median_cat_units = ['Median Category Units',median_cat_units]

            data_columns = [avg_total_spend,avg_total_units,median_total_spend,median_total_units,avg_item_price,median_item_price,avg_brand_spend,avg_brand_units,median_brand_spend,median_brand_units,avg_cat_spend,avg_cat_units,median_cat_spend,median_cat_units]

            columns = ['Metrics',v]
            for i in data_columns:
                d = pd.DataFrame([i],columns=columns)
                basket_overview = basket_overview.merge(d,on='Metric')
        
    basket_overview = basket_overview.transpose().reset_index()
    
    basket_overview['Average Share of Category Spend'] = basket_overview['Average Brand Spend'] / basket_overview['Average Category Spend']
    basket_overview['Average Share of Category Units'] = basket_overview['Average Brand Units'] / basket_overview['Average Category Units']
    basket_overview['Median Share of Category Spend'] = basket_overview['Median Brand Spend'] / basket_overview['Median Category Spend']
    basket_overview['Median Share of Category Units'] = basket_overview['Median Brand Units'] / basket_overview['Median Category Units']
        
    return basket_overview
    

In [41]:
def basket_adjacency(**params):
    
    brand_basket_adjacency = exec_presto(sql.cat_brand_trip.format(**params))
    category_basket_adjacency = exec_presto(sql.cat_category_trip.format(**params))
    product_basket_adjacency = exec_presto(sql.cat_product_trip.format(**params))
    
    df = basket
    cond = df.receipt_id.isin(cy.receipt_id)
    t_df = df[cond]
    
    brand_df = t_df.groupby('brand_name')
    brand_df = brand_df.agg({'receipt_id': 'nunique'})
    brand_df = brand_df.reset_index()
    brand_df['Total'] = brand_df['receipt_id']/len(t_df.receipt_id.unique()) 
    brand_df = [['brand_name','Total']]
    brand_basket_adjacency = brand_basket_adjacency.merge(brand_df,on='brand_name',how='inner')
    del brand_df
    
    category_df = t_df.groupby('category_name')
    category_df = category_df.agg({'receipt_id': 'nunique'})
    category_df = category_df.reset_index()
    category_df['Total'] = category_df['receipt_id']/len(t_df.receipt_id.unique()) 
    category_df = [['category_name','Total']]
    category_basket_adjacency = category_basket_adjacency.merge(category_df,on='category_name',how='inner')
    del cat_df
    
    product_df = t_df.groupby('category_name')
    product_df = product_df.agg({'receipt_id': 'nunique'})
    product_df = product_df.reset_index()
    product_df['Total'] = product_df['receipt_id']/len(t_df.receipt_id.unique()) 
    product_df = [['product_name','Total']]
    product_basket_adjacency = product_basket_adjacency.merge(product_df,on='product_name',how='inner')
    del product_df
    del t_df
    
    for c,v in seg_dict.iteritems():
        for v in v:
            seg = cy[cy[c] == v]
            cond = df.receipt_id.isin(seg.receipt_id)
            s_df = df[cond]
            del seg
            brand_df1 = s_df.groupby('brand_name')
            brand_df1 = brand_df1.agg({'receipt_id': 'nunique'})
            brand_df1 = brand_df1.reset_index()
            col_name = v
            brand_df1[col_name] = brand_df1['receipt_id']/len(s_df.receipt_id.unique()) 
            brand_df1 = [['brand_name',col_name]]

            category_df1 = s_df.groupby('category_name')
            category_df1 = category_df.agg({'receipt_id': 'nunique'})
            category_df1 = category_df.reset_index()
            category_df1[col_name] = category_df1['receipt_id']/len(s_df.receipt_id.unique()) 
            category_df1 = [['category_name',col_name]]

            product_df1 = s_df.groupby('category_name')
            product_df1 = product_df1.agg({'receipt_id': 'nunique'})
            product_df1 = product_df1.reset_index()
            product_df1[col_name] = product_df1['receipt_id']/len(s_df.receipt_id.unique()) 
            product_df1 = [['product_name',col_name]]
            
            brand_basket_adjacency = brand_basket_adjacency.merge(brand_df1,on='brand_name',how='outer')
            category_basket_adjacency = category_basket_adjacency.merge(category_df1,on='category_name',how='outer')
            product_basket_adjacency = product_basket_adjacency.merge(product_df1,on='product_name',how='outer')
    
    del brand_df1
    del category_df1
    del product_df1
    del s_df
    
    brand_basket_adjacency['Total Category Index'] = (brand_basket_adjacency['Total']/brand_basket_adjacency['Category Benchmark'])*100
    category_basket_adjacency['Total Category Index'] = (category_basket_adjacency['Total']/category_basket_adjacency['Category Benchmark'])*100
    product_basket_adjacency['Total Category Index'] = (product_basket_adjacency['Total']/product_basket_adjacency['Category Benchmark'])*100
    
    items = seg_dict.values()
    
    for x in items:
    
        cat_name = x+' Category Index'
        brand_basket_adjacency[cat_name] = (brand_basket_adjacency[x]/brand_basket_adjacency['Category Benchmark'])*100
        category_basket_adjacency[cat_name] = (category_basket_adjacency[x]/category_basket_adjacency['Category Benchmark'])*100
        product_basket_adjacency[cat_name] = (product_basket_adjacency[x]/product_basket_adjacency['Category Benchmark'])*100

        total_name = x+'Total Brand Index'
        brand_basket_adjacency[total_name] = (brand_basket_adjacency[x]/brand_basket_adjacency['Total'])*100
        category_basket_adjacency[total_name] = (category_basket_adjacency[x]/category_basket_adjacency['Total'])*100
        category_basket_adjacency[total_name] = (product_basket_adjacency[x]/product_basket_adjacency['Total'])*100
    
    
    return brand_basket_adjacency, category_basket_adjacency, product_basket_adjacency
    

In [42]:
def basket_product_overlap(**params):

    receipt_overlap = cy.groupby('global_product_name')
    receipt_overlap = receipt_overlap.agg({'receipt_id': 'nunique'})
    receipt_overlap = receipt_overlap.reset_index()
    receipt_overlap = receipt_overlap[['global_product_name','overlap']]
    receipt_overlap['overlap'] = receipt_overlap['receipt_id'] / len(receipt_overlap.receipt_id.unique())
    columns = ['global_product_name','Total']
    receipt_overlap.columns = cols
    
    products = list(cy.global_product_name.unique())
    
    for i in products:
        df = cy
        product = df[df['global_product_name'] == i]
        cond = df.receipt_id.isin(product.receipt_id)
        data = df[cond]
        data = data.groupby('global_product_name')
        data = data.agg({'receipt_id': 'nunique'})
        data = data.reset_index()
        data['overlap'] = data['receipt_id'] / len(data[data['global_product_name']==i].receipt_id.unique())
        data = data[['global_product_name','overlap']]
        name = i+' Overall'
        cols = ['global_product_name',name]
        data.columns = cols
        
        receipt_overlap = receipt_overlap.merge(data,on='global_product_name',how='outer')
    
    del data
    del product
    del df
    
    for c,v in seg_dict:
        for v in v:
                for i in products:
                    df = cy
                    product = df[(df['global_product_name'] == i) & (df[c] == v)]
                    cond = df.receipt_id.isin(product.receipt_id)
                    data = df[cond]
                    data = data.groupby('global_product_name')
                    data = data.agg({'receipt_id': 'nunique'})
                    data = data.reset_index()
                    data['overlap'] = data['receipt_id'] / len(data[(data['global_product_name']==i) & (data[c] == v)].receipt_id.unique())
                    data = data[['global_product_name','overlap']]
                    name = v+'-'+i
                    cols = ['global_product_name',name]
                    data.columns = cols
                    receipt_overlap = receipt_overlap.merge(data,on='global_product_name',how='outer')
    del data
    del product
    del df
    
    return receipt_overlap
            
        

In [43]:
def total_overlap(**params):
    
    overlap_dict = {}
    for c,v in seg_dict:
        for v in v:
            name = v+' Overlap'
            filt_df = cy[cy[c] == v]
            cond = cy.customer_id.isin(filt_df.customer_id)
            df = filt_df[cond]
            df = df.groupby(c)
            df = df.agg({'customer_id': 'nunique'})
            df = data.reset_index()
            df['overlap'] = df['customer_id'] / len(df[df[c]==v].customer_id.unique())
            df = df[[c,'overlap']]
            cols = [c,name]
            df.columns = cols
            overlap_dict[v] = df
    
    merged_overlap_dict = {}
    for k,v in seg_dict.items():
        n = seg_dict[k]
        temp_dict = {}
        for i in n:
            d = overlap_dict[i]
            temp_dict[i] = d
            loop_count = 1
            for i in n:
                df1 = temp_dict[i]
                if loop_count == 1:
                    data = df1
                    loop_count = loop_count+1
                else:
                    data = data.merge(df1,on=k,how='outer')
        merged_overlap_dict[k] = data
    overlap_df = pd.DataFrame()
    dfs_to_concat = list(merged_overlap_dict.keys())
    loop_countt = 1
    for i in dfs_to_concat:
        d = merged_overlap_dict[i]
        overlap_df = overlap_df.concat(d,axis=1)
    
    return overlap_df


In [50]:
def sa(segment_columns,
       segment_table: str = 'segment_table',
       name: str = 'project_name',
       segment_py_table: str = 'segment_py_table',
       category_table: str = 'category_table',
       category_py_table: str = 'category_py_table',
       active_table: str = 'active_table',
       active_py_table: str = 'active_py_table'):

    params = {}

    params['segment_columns'] = segment_columns
    params['segment_table'] = segment_table
    params['segment_py_table'] = segment_py_table
    params['category_table'] = category_table
    params['category_py_table'] = category_py_table
    params['active_table'] = active_table
    params['active_py_table'] = active_py_table
    params['name'] = name

    current_year_brand = exec_presto(sql.cy_brand.format(**params))
    past_year_brand = exec_presto(sql.cy_brand.format(**params))

    params['cy'] = current_year_brand
    params['py'] = past_year_brand
    
    cy = params['cy']
    py = params['py']
    
    seg_dict={}
    for i in params['segment_columns']:
        c = list(cy[i].unique())
        p = list(py[i].unique())
        common = set(c) - (set(c) - set(p))
        seg_dict[i] = common

    params['seg_dict'] = seg_dict

    total_pop_cy = exec_presto(sql.cy_total_pop.format(**params))
    total_pop_py = exec_presto(sql.py_total_pop.format(**params))
    total_pop_py = total_pop_py.total.sum()
    total_pop_cy = total_pop_cy.total.sum()
    

    params['total_pop_cy'] = total_pop_cy
    params['total_pop_py'] = total_pop_py

    tabs_data: list = []
    vol = volumetrics(**params)
    tabs_data.append([volume])
    
    del params['total_pop_cy']
    del params['total_pop_py']
    
    try_repeat = trial_repeat(**params)
    tabs_data.append([try_repeat])
    
    params['volume_overview'] = vol
    
    cat_cy = exec_presto(sql.cy_cat.format(**params))
    params['cy_cat'] = cat_cy
    del params['py']
    
    loyalty = brand_loyalty(**params)
    tabs_data.append([loyalty])
    

    basket = exec_presto(sql.basket_data.format(**params))
    cat_names = exec_presto(sql.category_names)
    brand_names = exec_presto(sql.brand_names)
    product_names = exec_presto(sql.product_names)
    basket = basket.merge(cat_names,on=secondary_category_id,how='left')
    basket = basket.merge(brand_names,on=brand_id,how='left')
    basket = basket.merge(product_names,on=global_product_id,how='left')
    params['basket'] = basket
    
    del cat_names
    del brand_names
    del product_names
    
    
    trip_overview = basket_overview(**params)
    tabs_data.append([trip_overview])    
    
    trip_adjacency = list(basket_adjacency(**params))
    tabs_data.append(trip_adjacency)
    
    trips_overlap = basket_product_overlap.format(**params)
    tabs_data.append([trip_overlap])
    
    segment_overlap = total_overlap(**params)
    tabs_list.append([segment_overlap])
        
    
    xlname = '{name}_Shopper_Analysis.xlsx'.format(**params)

    sheet_names = ["Volumentrics",
                   "Trial & Repeat",
                   "Brand Loyalty",
                   "Basket Overview",
                   "Basket Adjacency",
                   "Trip Overlap",
                   "Overall Overlap",
                   "Purchase History"]

    dump_to_excel(xlname, tabs_data, sheet_names=sheet_names)


In [69]:
sa(segment_columns=['segment'],segment_table='agajeski.whole_earth_portfolio_analysis',segment_py_table='agajeski.whole_earth_portfolio_analysis_py',category_table='agajeski.sugar_sweetener_category',category_py_table='agajeski.sugar_sweetener_category_py',active_table='agajeski.active_feb19_feb20',active_py_table='agajeski.active_feb20_feb21',name='test')

[2021-06-21 14:57:06] Request ID is: 10465589-1-1624309026.087. Please share it with Qubole Support team for any assistance
The Qubole application experienced an issue in processing this request and it was not successful. If you would like this issue investigated, please provide error code 90173f23-8007-4a66-ac84-6eb3f8291eca and precise details in a Qubole support ticket for further analysis.


ServerError: The Qubole application experienced an issue in processing this request and it was not successful. If you would like this issue investigated, please provide error code 90173f23-8007-4a66-ac84-6eb3f8291eca and precise details in a Qubole support ticket for further analysis.

In [ ]:
cy_brand = """
select
*
from {segment_table}
"""
py_brand = """
select
*
from {segment_py_table}
"""

cy_cat = """
select
*
from {category_table}
"""
py_cat = """
select
*
from {category_py_table}
"""

cy_total_pop = """
select
count(distinct customer_id) as total
from {active_table}
"""

py_total_pop = """
select
count(distinct customer_id) as total
from {active_py_table}
"""

basket_data = """
select
f.customer_id,
f.receipt_id,
f.brand_id,
f.global_product_id,
f.receipt_item_id,
f.secondary_category_id,
case when ri.ext_price is not null and ri.quantity > 1 then ri.ext_price
when ri.ext_price is null then ri.price
else ri.ext_price
end as "price"
from vw_fact_customer_receipt_item_procuct_details f
inner join {segment_table} s on f.receipt_id = s.receipt_id
where
((ri.ext_price is not null and ri.quantity > 1 and ri.ext_price > 0 and ri.ext_price < 50) or (ri.ext_price is null and ri.price > 0 and ri.price < 50) or (ri.ext_price > 0 and ri.ext_price < 50))
and f.verified is null
"""

brand_names = """
select
distinct name as brand_name,
id as brand_id
from vw_brands
"""
product_names = """
select
distinct name as product_name,
id as global_product_id
from vw_global_products
"""

category_names = """
select
distinct id as secondary_category_id,
name as category_name
from vw_product_categories
"""

cat_brand_trip = """
with total as(
select
count(distinct receipt_id) as total
from {category_table})
select distinct b.name as brand_name,
count(distinct f.receipt_id)/t.total as 'Category Benchmark'
from vw_fact_customer_receipt_item_product_details f,total t
join vw_brands b on b.id = f.brand_id
where
f.receipt_id in (select distinct receipt_id
from {category_table})
and f.verified is null
group by 1
order by 2 desc
"""

cat_category_trip = """
with total as(
select
count(distinct receipt_id) as total
from {category_table})
select distinct b.name as category_name,
count(distinct f.receipt_id)/t.total as 'Category Benchmark'
from vw_fact_customer_receipt_item_product_details f, total t
join vw_product_categories b on b.id = f.secondary_category_id
where
f.receipt_id in (select distinct receipt_id
from {category_table})
and f.verified is null
group by 1
order by 2 desc
"""

cat_product_trip = """
with total as(
select
count(distinct receipt_id) as total
from {category_table})
select distinct b.name as product_name,
count(distinct f.receipt_id)/t.total as 'Category Benchmark'
from vw_fact_customer_receipt_item_product_details f,total t
join vw_global_products b on b.id = f.global_product_id
where
f.receipt_id in (select distinct receipt_id
from {category_table})
and f.verified is null
group by 1
order by 2 desc
"""